In [5]:
import pandas as pd
import numpy as np
import unicodedata
from edastatmil_milser import edas_tatmil as EDA
from szip_engine import compress_and_split, decompress_volumes
from tools import remove_spaces, remove_tabs, replicates_rows,rename_clusters, nulls_columns, nulls_in_cluster, split_clusters
#pd.set_option('display.max_rows', None)

En este notebook vamos a realiar las siguientes tareas:
1. Construir nuestro dataset de inicio (punto 1 de Apuntes).
2. Hacer la limpieza preliminar de los datos (punto 2.1 de Apuntes).
   
Las funciones que necesitamos están descritas en tool.py para que este notebook sea más legible.

## 1. Stars sample preview

Partimos de un dataset que se ha construido tal y como se describe en el punto 1 de los Apuntes: descargando samples de datos para diversos trabajos científicos relacionados con cúmulos de estrellas y unificándolos bajo 5 columnas:

In [2]:
df = pd.read_csv('../Samples/Clusters.csv')
df.head()

,_Glon,_Glat,_RAJ2000,_DEJ2000,Cluster
0,136.163369,-23.291857,28.329417,37.984833,Mel12
1,136.390938,-23.836403,28.405333,37.404806,Mel12
2,136.386537,-23.211326,28.607250,38.009139,Mel12
3,136.553405,-23.705483,28.629333,37.492111,Mel12
4,136.587727,-23.750297,28.652583,37.440583,Mel12


In [3]:
df.shape

(992227, 5)

Estas columnas se corresponden con las coordenadas de las estrellas en dos sistemas de coordenadas diferentes (coordenadas galácticas y ecuatoriales) y una columna con el nombre que tenía el cúmulo en el trabajo científico del cual proviene el dato. Cada fila se corresponte, por tanto, con una estrella.

Obtenemos una lista de los numbres de los clusters incluidos en el stars sample

In [4]:
actual_name = df['Cluster'].unique()
actual_name

array(['Mel12', 'Mel25', 'NGC 2632', 'NGC 2168', 'NGC 2099', 'NGC 1746',
       'NGC 1039', 'NGC 1647', 'NGC 2183', 'NGC 1912', 'NGC 2281',
       'NGC 2420', 'NGC 2158', 'NGC 869', 'NGC 1662', 'Basel 11B',
       'NGC 1528', 'NGC 2252', 'Basel 4', 'NGC 1960', 'NGC 6205',
       'Waterloo 2', 'NGC 6819', 'NGC 5272', 'Berkeley 71', 'Berkeley 32',
       'Berkeley 17', 'Berkeley 29', 'Berkeley 53', 'Berkeley 66',
       'FSR 0494', 'IC 166', 'King 5', 'King 7', 'M 107', 'M 13', 'M 15',
       'M 2', 'M 3', 'M 35', 'M 5', 'M 53', 'M 67', 'M 71', 'M 92',
       'NGC 1245', 'NGC 1798', 'NGC 188', 'NGC 2243', 'NGC 2682',
       'NGC 4147', 'NGC 5466', 'NGC 6791', 'NGC 6811', 'NGC 7789',
       'Pleiades', 'Teutsch 51', 'Mel13', 'Mel14', 'NGC 2532', 'IC 2395',
       'NGC 2808', 'NGC 0104', 'NGC 1904', 'NGC 1851', 'NGC 6266',
       'NGC 5139', 'NGC 3201', 'NGC 0362', 'NGC 5286', 'NGC 6218',
       'NGC 5904', 'NGC 6093', 'NGC 6254', 'NGC 6293', 'NGC 6388',
       'NGC 6522', 'NGC 6441', 'NGC

Hay que comprobar que no tenemos el mismo cluster nombrado de dos formas diferentes. Para ello:
1. Vamos a hacer una tabla con los diferentes nombres de los clusters los diferentes catálogos posibles.
2. Escogeremos el catálogo con el que vamos a trabajar para tener todos los cluster identificados bajo un mismo criterio.
3. Renombraremos los clusters con el nombre escogido.

### 1.1 Rename clusters
Se hace una lista para los nombres en cada catalogo, basada en la lista de nombres de clusters obtenida del stars sample. Si un nombre de la lista inicial no está en algún catálogo, agregamos None.

In [5]:
NGC_names = ['NGC752', None, 'NGC 2632', 'NGC 2168', 'NGC 2099', 'NGC 1746', 'NGC 1039','NGC 1647','NGC 2183','NGC 1912', 'NGC 2281', 'NGC 2420', 'NGC 2158','NGC 869','NGC 1662', None,'NGC 1528','NGC 2252', None,'NGC 1960', 'NGC 6205', None, 'NGC 6819','NGC 5272', None, None, None, None, None, None, None, None, None, None, 'NGC 6171', 'NGC 6205','NGC 7078', 'NGC 7089', 'NGC 5272',' NGC 2168', 'NGC 5904','NGC 5024','NGC 2682','NGC 6838','NGC 6341','NGC 5145','NGC 1798','NGC 188','NGC 2243','NGC 2682','NGC 4147','NGC 5466','NGC 6791','NGC 6811','NGC 7789', None, None,'NGC 869', 'NGC 884','NGC 2532', None, 'NGC 2808','NGC 0104','NGC 1904','NGC 1851','NGC 6266', 'NGC 5139','NGC 3201','NGC 0362','NGC 5286', 'NGC 6218', 'NGC 5904', 'NGC 6093', 'NGC 6254', 'NGC 6293', 'NGC 6388','NGC 6522', 'NGC 6441', 'NGC 6541','NGC 6624','NGC 7078','NGC 6656','NGC 6681','NGC 6752','NGC 7089','NGC 7099', None, None, None,'NGC 457','NGC 884','NGC 1502','NGC 2244', 'NGC 2362','NGC 2384','NGC 2422','NGC 2467','NGC 3293','NGC 4755','NGC 6193','NGC 7160', None, None, None,'NGC 1261','NGC 3201','NGC 6139','NGC 6205','NGC 6362','NGC 6397','NGC 6712', None]
Mel_names = ['Mel 12', 'Mel 25', 'Mel 88', None, None, None, None, None, None, None, None, 'Mel 69', 'Mel 40', 'Mel 13', None, None, None, None, None, None, None, None, 'Mel 223', 'Mel 119', None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, 'Mel 18', None, 'Mel 2','Mel 46', None, None, None, None, 'Mel 222', 'Mel 245', 'Mel 22', None, 'Mel 13', 'Mel 14', None, None, None, 'Mel 1', None, None, None, None, None, 'Mel 4', None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, 'Mel 15', None, None, 'Mel 14', None, None, None, None, None, None,'Mel 100','Mel 144', None, None, None, None, None, None, None, None, None, None, None, None, None]
M_names = [None, None, 'M 44', 'M 35', 'M 37', None, 'M 34',None, None, 'M 38', None, None, None, None, None, None, None, None, None, 'M 36', 'M 13', None, None, 'M 3', None, None, None, None, None, None, None, None, None, None, 'M 107', 'M 13','M 15','M 2', 'M 3','M 35','M 5', 'M 53', 'M 67', 'M 71', 'M 92', None, None, None, None, 'M 67', None, None, None, None, None,'M 45', None, None, None, None, None, None, None, 'M 79', None, 'M 62', None, None, None, None, 'M 12', 'M 5', 'M 80', 'M 10', None, None, None, None, None, None,'M 15', 'M 22','M 70', None,'M 2','M 30', None, None, None, None, None, None, None, None, None, 'M 47', None, None, None, None, None, None, None, None, None, None, None, 'M 13', None, None, None, None]
Common_name = [None, 'Hiades', 'Beehive', None, None, None, None, None, None, None, None, None, None, 'H Persei', None, 'Basel 11b', None, None, None, None,'Hercules', None, 'Foxhead', None, None, None, None, None, None, None, None, None, None, None, None, 'Hercules', None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, 'Pleiades', None, 'H Persei', 'Chi Persei', None, None, None, '47 Tuc', None, None, None, 'Ome Cen', None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, 'AFGL 333', 'lam Cen','ET','Chi Persei', None, None, 'Tau CMa', None, None, None, None,'Jewel Box', None, None, None, None, None, None, None, None,'Hercules', None, None, None, None]

Comprobamos que la longitud de todas las listas es la misma, de forma que no nos hemos equivocado. Si todo está bien, construimos el dataframe de la tabla de equivalencia de nombres.

In [6]:
print(len(actual_name))
print(len(NGC_names))
print(len(Mel_names))
print(len(M_names))
print(len(Common_name))

112
112
112
112
112


In [7]:
clusters_names = pd.DataFrame(actual_name, columns=['Actual Names'])
clusters_names['NGC_names'] = NGC_names
clusters_names['Mel_names'] = Mel_names
clusters_names['M_names'] = M_names
clusters_names['Common_name'] = Common_name
clusters_names.head(5)

,Actual Names,NGC_names,Mel_names,M_names,Common_name
0,Mel12,NGC752,Mel 12,None,None
1,Mel25,None,Mel 25,None,Hiades
2,NGC 2632,NGC 2632,Mel 88,M 44,Beehive
3,NGC 2168,NGC 2168,None,M 35,None
4,NGC 2099,NGC 2099,None,M 37,None


Eliminamos los espacios de los nombres

In [8]:
clusternames_cols = clusters_names.columns.tolist()

for col in clusternames_cols:
    clusters_names[col] = clusters_names[col].apply(remove_spaces)
clusters_names.head(5)

,Actual Names,NGC_names,Mel_names,M_names,Common_name
0,Mel12,NGC752,Mel12,None,None
1,Mel25,None,Mel25,None,Hiades
2,NGC2632,NGC2632,Mel88,M44,Beehive
3,NGC2168,NGC2168,None,M35,None
4,NGC2099,NGC2099,None,M37,None


Eliminamos las filas repetidas usando la columna NGC_names, ya que es donde más nombres encontramos. Hay que tener en cuenta que no se aplique la eliminación de duplicados a las filas donde NGC_names sea None

In [9]:
Name_equival = replicates_rows(clusters_names, 'NGC_names')
Name_equival.head(5)

,Actual Names,NGC_names,Mel_names,M_names,Common_name
0,Mel12,NGC752,Mel12,None,None
1,NGC2632,NGC2632,Mel88,M44,Beehive
2,NGC2168,NGC2168,None,M35,None
3,NGC2099,NGC2099,None,M37,None
4,NGC1746,NGC1746,None,None,None


In [10]:
Name_equival.to_csv('../Samples/Clean/NamesCatalogEquivalence.csv')

Trabajaremos siempre que sea posible con el NGC_name. En los casos en los que no tenemos NGC_name, se trabajará con el Actual Name.Entonces:
1. Tanto en la columna Cluster del dataset como en las columnas de la tabla clusters_names, eliminaremos los espacios.
2. Recorremos el dataset de estrellas y allá donde la columna Cluster no tiene el nombre NGC, lo tomaremos de la tabla de equivalencia de nombres y lo sustituiremos. Si no hay un nombre NGC, dejamos el que está.

In [11]:
df['Cluster'] = df['Cluster'].str.replace(r'\s+', '', regex=True).str.strip()

In [12]:
df['Cluster'] = df['Cluster'].apply(rename_clusters)
actual_name = df['Cluster'].unique()
actual_name

array(['NGC752', nan, 'NGC2632', 'NGC2168', 'NGC2099', 'NGC1746',
       'NGC1039', 'NGC1647', 'NGC2183', 'NGC1912', 'NGC2281', 'NGC2420',
       'NGC2158', 'NGC869', 'NGC1662', 'NGC1528', 'NGC2252', 'NGC1960',
       'NGC6205', 'NGC6819', 'NGC5272', 'NGC6171', 'M13', 'NGC7078',
       'NGC7089', 'M3', 'M35', 'NGC5904', 'NGC5024', 'NGC2682', 'NGC6838',
       'NGC6341', 'NGC1245', 'NGC1798', 'NGC188', 'NGC2243', 'NGC4147',
       'NGC5466', 'NGC6791', 'NGC6811', 'NGC7789', 'Mel13', 'NGC884',
       'NGC2532', 'NGC2808', 'NGC0104', 'NGC1904', 'NGC1851', 'NGC6266',
       'NGC5139', 'NGC3201', 'NGC0362', 'NGC5286', 'NGC6218', 'NGC6093',
       'NGC6254', 'NGC6293', 'NGC6388', 'NGC6522', 'NGC6441', 'NGC6541',
       'NGC6624', 'NGC6656', 'NGC6681', 'NGC6752', 'NGC7099', 'NGC457',
       'NGC1502', 'NGC2244', 'NGC2362', 'NGC2384', 'NGC2422', 'NGC2467',
       'NGC3293', 'NGC4755', 'NGC6193', 'NGC7160', 'NGC1261', 'NGC6139',
       'NGC6362', 'NGC6397', 'NGC6712'], dtype=object)

Además, buscando en bases de datos los nombres de los clusters, hemos visto que los clusters FSR 0494 y CR 228 no existen como clusters oficiales, así que eliminaremos las filas correspondientes a estos clusters del dataset.

In [13]:
nonfinderdlusters = ['FSR0494','CR228']
df = df.query('Cluster not in @nonfinderdlusters')

In [14]:
df.to_csv('../Samples/RenamedClusters.csv')
df.head(5)

,_Glon,_Glat,_RAJ2000,_DEJ2000,Cluster
0,136.163369,-23.291857,28.329417,37.984833,NGC752
1,136.390938,-23.836403,28.405333,37.404806,NGC752
2,136.386537,-23.211326,28.607250,38.009139,NGC752
3,136.553405,-23.705483,28.629333,37.492111,NGC752
4,136.587727,-23.750297,28.652583,37.440583,NGC752


In [15]:
n,c = EDA.explore(df)

Rows: 992227
Columns: 5
+----------+----------------+------------+-----------+---------------+
|          | Non-Null Count | Null Count | Data Type | Data Category |
+----------+----------------+------------+-----------+---------------+
|  _Glon   |     992225     |     2      |  float64  |   Numerical   |
|  _Glat   |     992225     |     2      |  float64  |   Numerical   |
| _RAJ2000 |     992225     |     2      |  float64  |   Numerical   |
| _DEJ2000 |     992225     |     2      |  float64  |   Numerical   |
| Cluster  |     990893     |    1334    |  object   |  Categorical  |
+----------+----------------+------------+-----------+---------------+


Ahora con este dataframe haremos un merge en la base de datos de Gaia DR3 mediante el software TopCat. Lo que hace este software es:
1. Accede a la base de datos de la misión espacial Gaia, en su versión más actualiada (DR3).
2. Usando las coordenadas que situan a cada estrella en el cielo (_RAJ2000 y _DEJ2000 en nuestro dataset), va a buscar nuestras estrellas en la base de datos.
3. Agregará para cada estrella las columnas con todos los datos y características que tenga sobre ellas. 

>El dataset resultante pesa mucho asi que esta comprimido en distintas partes. Para obtenerlo hay que descomprimirlos en la carpeta ../Samples/

In [16]:
#decompress_volumes('STARSSample', '../Samples', '../Samples/Compressed') TODO descomentar

# Alternativamente el dataset esta en esta carpeta de drive:
# https://drive.google.com/drive/folders/1ac8yV0D-KNRFOH3jnlYUnNfeU6bkZc8S

## 2. ClustersInfo explore

Como se ha visto que faltan datos para algunas de las estrellas, vamos a intentar obtener la información que nos falta de otras fuentes. Para ello contruímos el dataset ClustersInfo. Esto se hace a partir de la base de datos SIMBAD, que es una base de datos con información sobre muchos de los objetos del cielo. 
1. Se buscan los cúmulos que tenemos en nuestro dataset.
2. Se toma la información disponible y se construye el dataset.

De esta forma, ClustersInfo contiene información general sobre el cúmulo. Es importante ya que con ella vamos a poder reemplazar información que falte en STARSSample e identificar los clusters que obtengamos.

In [17]:
clusters_info = pd.read_csv('../Samples/ClustersInfo.csv', delimiter=';')
clusters_info.head(5)

,Cluster,RAJ2000,DEJ2000,GLON,GLAT,RV (km/s),pmRA (mas/yr),pmDE (mas/yr),Plx (mas)
0,NGC 752,01 56 53.5,37 47 38,136.958,-23.28,5.44,98.092,-117.637,22.304
1,Mel25,04 29 47.3,16 56 53,179.627,-21.124,37.97,104.920,-28.000,21.052
2,NGC 2632,08 40 13.0,19 37 16,205.952,32.427,34.94,-36.047,-12.917,5.371
3,NGC 2168,06 09 05.3,24 20 10,186.609,2.230,-6.37,22.784,-29.336,11.301
4,NGC 2099,05 52 17.8,32 32 42,177.64,3.086,7.70,1.924,-5.648,666.000


Tenemos indicativos de tabulador en algunas celdas. Limpiamos esto:

In [18]:
clusters_info = clusters_info.map(remove_tabs)
clusters_info.head(5)

,Cluster,RAJ2000,DEJ2000,GLON,GLAT,RV (km/s),pmRA (mas/yr),pmDE (mas/yr),Plx (mas)
0,NGC 752,01 56 53.5,37 47 38,136.958,-23.28,5.44,98.092,-117.637,22.304
1,Mel25,04 29 47.3,16 56 53,179.627,-21.124,37.97,104.920,-28.000,21.052
2,NGC 2632,08 40 13.0,19 37 16,205.952,32.427,34.94,-36.047,-12.917,5.371
3,NGC 2168,06 09 05.3,24 20 10,186.609,2.230,-6.37,22.784,-29.336,11.301
4,NGC 2099,05 52 17.8,32 32 42,177.64,3.086,7.70,1.924,-5.648,666.000


Hay que hacer el mismo tratamiento para los nombres que con STARSSample:
1. Eliminar espacios
2. Sustituir por los nombres que hemos dejado en STARSSample

In [19]:
clusters_info['Cluster'] = clusters_info['Cluster'].str.replace(r'\s+', '', regex=True).str.strip()

In [20]:
clusters_info['Cluster'] = clusters_info['Cluster'].apply(rename_clusters)
clusters_info.head(5)

,Cluster,RAJ2000,DEJ2000,GLON,GLAT,RV (km/s),pmRA (mas/yr),pmDE (mas/yr),Plx (mas)
0,NGC752,01 56 53.5,37 47 38,136.958,-23.28,5.44,98.092,-117.637,22.304
1,NaN,04 29 47.3,16 56 53,179.627,-21.124,37.97,104.920,-28.000,21.052
2,NGC2632,08 40 13.0,19 37 16,205.952,32.427,34.94,-36.047,-12.917,5.371
3,NGC2168,06 09 05.3,24 20 10,186.609,2.230,-6.37,22.784,-29.336,11.301
4,NGC2099,05 52 17.8,32 32 42,177.64,3.086,7.70,1.924,-5.648,666.000


In [21]:
# Rename columns to eliminate ()
clusters_info = clusters_info.rename(columns=lambda x: x.split('(')[0].strip())

# Add distance calculate from parallax (D=1000/|Plx|)
clusters_info['Dist'] = 1000 / clusters_info['Plx'].abs()

# Drop duplicates
clusters_info_clean = clusters_info.drop_duplicates(subset='Cluster')

# Drop unused columns
clusters_info_clean = clusters_info_clean.drop(columns=['RAJ2000','DEJ2000','GLON','GLAT'])

In [22]:
# Save
clusters_info_clean.to_csv('../Samples/Clean/ClustersInfo_clean.csv')

## 3. EDA I: STARSSample explore

Una vez que tenemos nuestro sample completo de estrellas y tenemos el sample de clusters del cual podemos obtener información, vamos a comenzar el EDA.

En primer lugar haremos un tratamiento general de los datos, centrado en eliminar columnas inncecesarias y valores nulos.

In [24]:
stars_df = pd.read_csv('../Samples/STARSSample.csv')
num, cat = EDA.explore(stars_df)
print('Do it')

Rows: 984102
Columns: 134
+----------------+----------------+------------+-----------+---------------+
|                | Non-Null Count | Null Count | Data Type | Data Category |
+----------------+----------------+------------+-----------+---------------+
|      col1      |     984102     |     0      |   int64   |   Numerical   |
|     _Glon      |     984102     |     0      |  float64  |   Numerical   |
|     _Glat      |     984102     |     0      |  float64  |   Numerical   |
|    _RAJ2000    |     984102     |     0      |  float64  |   Numerical   |
|    _DEJ2000    |     984102     |     0      |  float64  |   Numerical   |
|    Cluster     |     984102     |     0      |  object   |  Categorical  |
|    DR3Name     |     984102     |     0      |  object   |  Categorical  |
|     RAdeg      |     984102     |     0      |  float64  |   Numerical   |
|     DEdeg      |     984102     |     0      |  float64  |   Numerical   |
|   errHalfMaj   |     984102     |     0      |  

Definición de las columnas:
- col1: índice
- _Glon: coordenada longitud galáctica. Ángulo sobre el disco de la galaxia desde la dirección del centro galáctico hasta la vertical de la estrella.
- _Glat: coordenada latitud galáctica. Ángulo desde el disco galactico hasta la estrella.
- _RAJ2000: coordenada ascensión recta de la época 2000. Ángulo sobre el ecuador desde el punto vernal hasta la vertical de la estrella.
- _DEJ2000: coordenada declinación de la época 2000. Ángulo desde el ecuador hasta la estrella.
- Cluster: Nombre del cúmulo al que pertenece la estrella.
- DR3Name: Nombre de la estrella en el catálogo DR3 de Gaia.
- RAdeg: igual que _RAJ2000
- DEdeg: igual que _DEJ2000
- *errHalfMaj, *errHalfMin, *errPosAng: errores de la medida de la posición.
- *SolID, *Source, *RandomI: otros identificadores.
- e_RAdeg, e_DEdeg: errores en la medida de ascensión recta y declinación.
- Plx: paralaje. Es el dato que necesitamos para obtener la distancia.
- e_Plx: error en la medida del paralaje.
- RPlx: paralaje dividido por su desviación estándar.
- PM: movimiento propio.
- pmRA: velocidad de la estrella en su movimiento propio respecto al centro de la galaxia en la coordenada ascensión recta. Medido en mas/yr.
- e_pmRA: error en la medida de pmRA.
- pmDE: velocidad de la estrella en su movimiento propio respecto al centro de la galaxia en la coordenada declinación. Medido en mas/yr.
- e_pmDE: error en la medida de pmDE.
- RADEcor, RAPlxcor,RApmRAcor,RApmDEcor, DEPlxcor, DEpmRAcor,DEpmDEcor, PlxpmRAcor,PlxpmDEcor, pmRApmDEcor: correlaciones entre magnitudes.
- NAL, NAC: número total de observaciones en diferentes longitudes de onda.
- NgAL, NgAL: número de buenas observaciones en diferentes longitudes de onda.
- *gofAL, *chi2AL: estádisticas de la bondad de las observaciones.
- epsi, sepsi: exceso de ruido en la observación de la fuente y su significancia.
- *Solved: parámetros astrométricos resueltos.
- *APF: astrometría primaria o secundaria.
- *nueff: número de onda efectivo usado en la solución astrométrica.
- pscol: pseudocolor estimado astrométricamente.
- e_pscol: error en la medida de pscol.
- RApscolCorr, DEpscolCorr, PlxpscolCorr,pmRApscolCorr,pmDEpscolCorr: correlaciones entre las variables.
- *MatchObsA: tránsitos encontrados en FOV usando la solución AGIS.
- Nper: número de periodos de visibilidad usados en la solución astrométrica.
- amax: eje semimayor del elipsoide de error.
- *IPDgofha: amplitud del GoF IPD.
- MatchObs: número total de tránsitos.
- *IPDgofhp: fase del GoF IPD.
- *IPDfmp: porcentaje de éxito de la ventana IPD.
- *IPDfow: porcentaje de tránsitos fallidos.
- RUWE: error de peso normalizado.
- *Dup: si es una fuente con múltiples identificadores.
- o_Gmag: número de tránsitos que contibuyen a la medida del flujo en Gmag.
- FG: flujo medio en la banda G.
- e_FG: error de FG.
- RFG: FG dividido por su error.
- Gmag: magnitud en la banda G.
- e_Gmag: error de Gmag.
- o_BPmag: numero de observaciones que contribuye a fotometría BP.
- FBP: flujo medio en la banda BP.
- e_FBP: error de FBP.
- RFBP: FBP dividido por su error.
- BPmag: magnitud en la banda BP.
- e_BPmag: error de BPmag.
- o_RPmag: numero de tránsitos que contribuye a la fotometría RP.
- FRP: flujo medio en la banda RP.
- e_FRP: error de FRP.
- RFRP: FRP dividido por su error.
- RPmag: magnitud en la banda RP.
- e_RPmag: error de RPmag.
- E(BP/RP): factor de exceso de color en BP/RP.
- NBPcont, NRPcont: numero de tránsitos contaminados en cada banda.
- NBPblend, NRPblend: número de tránsitos mezclados en cada banda.
- *Mode: modo de procesamiento de la fotometría.
- BP-RP, BP-G,G-RP: colores en diferentes bandas.
- RV: velocidad radial.
- e_RV: error de la velocidad radial.
- *n_RV: método usado para obtener la velocidad radial.
- o_RV: número de tránsitos usados para obtener la velocidad radial.
- o_RVd: número de tránsitos válidos.
- RVNper: número de periodos de visibilidad usados para obtener la velocidad radial.
- RVS/N: ratio de ruido en la señal espectral para obtener la velocidad radial.
- *RVgof, *RVchi2: estadísticas en el cálculo de RV.
- *RVTdur: tiempo cubierto en la serie temporal de velocidad radial.
- *RVamp: amplitud máxima en la serie temporal de velocidad una vez eliminados los outliers.
- RVtempTeff: temperatura efectiva del modelo usado para calcular la velocidad radial.
- RVtemplogg: logaritmo de la temperatura efectiva.
- RVtemp[Fe/H]: metalicidad del modelo usado para calcular la temperatura efectiva.
- Vatmparam: origen de los parámetros atmoféricos asociados al modelo.
- vbroad: líneas espectrales ampliadas.
- e_Vbroad: error de vbroad.
- o_Vbroad: número de tránsitos usados para calcular vbroad.
- GRVSmag: magnitud Grvs.
- e_GRVSmag: error de GRVSmag.
- o_GRVSmag: número de tránsitos usados para calcular o_GRVSmag.
- RVSS/N: ratio de señal/ruido en el espectro de RVS.
- *VarFlag: marcador de variabilidad fotométrica.
- PQSO: probabilidad de que sea un quásar.
- PGal: probabilidad de que sea una galaxia.
- PSS: probabilidad de que sea una única estrella.
- Teff: temperatura efectiva con la banda BP/RP.
- logg: gravedad en la superficie de la estrella con la banda BP/RP
- \[Fe/H]: metalicidad
- Dist: distancia usando la banda BP/RP.
- A0: extinción.
- AG: extinción en la banda G.
- E(BP-RP): enrojecimiento en la banda BP-RP.
- *Lib: nombre de la librería usada en parámetros de astrofotometría.
- RAJ2000: igual que _RAJ2000
- DEJ2000: igual que _DEJ2000
- *e_RAJ2000: error de RAJ2000
- *e_DEJ2000: error deDEJ2000
- *RADEcorj2000: corrección de época J2000
- angDist: distancia angular.
- *CartessianCoord: vector con las coordenadas cartesianas x,y,z
- X: coordenada cartesiana x
- Y: coordenada cartesiana y
- Z: coordenada cartesiana z
  
> Nuestro target es la columna Clusters y el resto son variables predictoras

### 3.1 Eliminación de las columnas repetidas


In [25]:
stars_df_clean = stars_df.drop(columns=['RAdeg','DEdeg','RAJ2000','DEJ2000'])
stars_df_clean.loc[stars_df_clean['Cluster'].str.contains('NGC 869'), 'Cluster'] = 'NGC869'
stars_df_clean.loc[stars_df_clean['Cluster'].str.contains('NGC 752'), 'Cluster'] = 'NGC752'
stars_df_clean.loc[stars_df_clean['Cluster'].str.contains('NGC 884'), 'Cluster'] = 'NGC884'

stars_df_clean.shape

(984102, 130)

### 3.2 Eliminación de información irrelevante

En nuestra experiencia,sabemos que las columnas marcadas en la definición anterior con * son irrelevantes para nuestro análisis y las quitamos.

In [26]:
stars_df_clean = stars_df_clean.drop(columns=['DR3Name','errHalfMaj', 'errHalfMin', 'errPosAng', 'SolID', 'Source', 'RandomI', 'gofAL', 'chi2AL', 'Solved', 'APF', 'nueff', 'MatchObsA', 'IPDgofha', 'IPDgofhp', 'IPDfmp', 'IPDfow', 'Dup', 'Mode', 'n_RV', 'RVgof', 'RVchi2', 'RVTdur', 'RVamp', 'VarFlag', 'Lib', 'e_RAJ2000','e_DEJ2000','e_DEJ2000','RADEcorJ2000', 'CartesianCoord'])
stars_df_clean.shape

(984102, 100)

### 3.3 Uso de información para eliminar filas erróneas

Las variables PQSO y PGal nos dan las probabilidad que estima el satélite de que ese objeto sea en realidad una galaxia lejana o un quásar en lugar de una estrella. Usaremos esta información para eliminar las estrellas con una alta probabilidad de no serlo. En concreto, quitaremos aquellas filas que con una probabilidad mayor que 10% no son estrellas.

In [27]:
# Drop 'PQSO'> 0.05 rows
stars_df_clean = stars_df_clean[stars_df_clean['PQSO'] <= 0.1]

# Drop 'PGal' > 0.05
stars_df_clean = stars_df_clean[stars_df_clean['PGal'] <= 0.1]

stars_df_clean.shape

(874843, 100)

### 3.4 Tratamiento de valores nulos
Vamos a hacer una exploración para ver donde tenemos valores nulos:

In [28]:
null_counts = stars_df_clean.isna().sum()
null_counts

col1             0
_Glon            0
_Glat            0
_RAJ2000         0
_DEJ2000         0
             ...  
E(BP-RP)    587949
angDist          0
X            73629
Y            73629
Z            73629
Length: 100, dtype: int64

#### 3.4.1 Distancia y paralaje
Si la estrella tiene Plx no nulo, entonces podemos calcular Dist como:

Dist = 1000/abs(Plx)

In [29]:
stars_df_clean.loc[stars_df_clean['Dist'].isna(), 'Dist'] = 1000 / stars_df_clean['Plx'].abs()

Algunos de los valores nulos que tenemos podemos sustituirlos por los valores generales para los cúmulos que tenemos en ClustersInfo_clean. 

Vamos a darles el valor por defecto del cluster a las estrellas que no tienen su dato propio de la siguiente forma:
| Columna en STARSSample | Columna en ClusterInfo_clean |
|------------------------|-----------------------------|
| Cluster                | Cluster                     |
| RV                     | RV                          |
| Plx                    | Plx                         |
| pmRA                   | pmRA                        |
| pmDE                   | pmDE                        |
| Dist                   | 1000/abs(Plx)               |

In [30]:
clusters_info_clean = pd.read_csv('../Samples/Clean/ClustersInfo_clean.csv')

In [31]:
stars_df_clean.shape

(874843, 100)

In [32]:
# Merge STARTSSample and ClustersInfo_clean by 'Cluster' column
merged_df = pd.merge(stars_df_clean,clusters_info_clean, on='Cluster', suffixes=('_stars','_clusters'), how='left', validate='many_to_one')

# Fill null valores in each columna by the corresponding value in ClustersInfo
columns_to_fill = ['RV', 'Plx', 'pmRA', 'pmDE', 'Dist']
for column in columns_to_fill:
    merged_df[column + '_stars'].fillna(merged_df[column + '_clusters'])

# Delete ClustersInfo columns
merged_df = merged_df.drop(columns=[col + '_clusters' for col in columns_to_fill])

# Rename STARSSample columns to eliminarte the suffix
merged_df.rename(columns={col + '_stars': col for col in columns_to_fill}, inplace=True)

# Refilled
stars_df_clean = merged_df

In [33]:
stars_df_clean.shape

(874843, 101)

La estrellas que quedan sin medidade paralaje se eliminan del dataset, pues ya no tenemos otras formas de obtener este dato. 

Una sustitución por alguna métrica estadística introducirá errores en el modelo, ya que a partir del paralaje se obtiene la distancia, que será clave para determinar si una estrella pertenece o no a un cúmulo. Como no son muchas en porporción al total de filas, las eliminamos.

In [34]:
stars_df_clean = stars_df_clean.dropna(subset=['Plx'])
stars_df_clean.shape

(801214, 101)

#### 3.4.2 Extinción y enrojecimiento
Vamos a igualar a 0 las columnas: A0, AG y E(BP-RP), si son nulas. Esto significa que consideraremos que entre la Tierra y esas estrellas el medio interestelar es transparente.Es decir, que no hay polvo ni gas que absorba parte de la luz de la estrella ni la enrrojezca. 

Es un abordaje habitual en astrofísica cuando no se dispone de estos datos.


In [35]:
stars_df_clean.loc[:, 'A0'] = stars_df_clean['A0'].fillna(0)
stars_df_clean.loc[:, 'AG'] = stars_df_clean['AG'].fillna(0)
stars_df_clean.loc[:, 'E(BP-RP)'] = stars_df_clean['E(BP-RP)'].fillna(0)
stars_df_clean.shape

(801214, 101)

#### 3.4.3 Nulos derivados
Comprobamos ahora cuales son las columnas con más de un 50% de valores nulos.

In [36]:
cols_50pc_nulls = nulls_columns(stars_df_clean, 50)
cols_50pc_nulls 

pscol            53.072088
e_pscol          53.072088
RApscolCorr      53.072088
DEpscolCorr      53.072088
PlxpscolCorr     53.072088
pmRApscolCorr    53.072088
pmDEpscolCorr    53.072088
RV               97.416296
e_RV             97.416296
o_RV             97.416296
o_RVd            97.416296
RVNper           97.416296
RVS/N            97.416296
RVtempTeff       97.416296
RVtemplogg       97.416296
RVtemp[Fe/H]     97.416296
Vatmparam        97.416296
vbroad           99.646536
e_Vbroad         99.646536
o_Vbroad         99.646536
GRVSmag          97.525879
e_GRVSmag        97.525879
o_GRVSmag        97.525879
RVSS/N           99.825889
Teff             64.192588
logg             64.192588
[Fe/H]           64.192588
dtype: float64

Vemos que muchas de las columnas con nullos realmente provienen de que son columnas calculadas a partir de una de origen nulo. Esto ocurre con:
- e_RV, o_RV,o_RVd, RVper,RVS/N, RVtempTeff, RVtemlogg,RVtemp[FE/H] y Vatmparam, que provienen de nulos en RV
- e_Vbroad, o_Vbroad, que provienen de nulos en vbroad
- e_GRVSmag, o_GRVSmag, que provienen de nulos en GRVSmag
- Teff y logg, provienen de nulos en [Fe/H]
- e_pscol, RApscolCorr, DEpscolCorr, PlxpscolCorr, pmRApscolCorr, pmDEpscolCorr, que provienen de nulos en pscol 

De esta columnas, no tenemos cómo obtener más información y, tal y como hemos explicado antes, una sustitución por métricas estadísticas no sería apropiada. Como, en nuestra experiencia noson columas muy importantes, las eliminamos:

In [37]:
stars_df_clean = stars_df_clean.drop(columns=cols_50pc_nulls .keys())
stars_df_clean.shape

(801214, 74)

In [38]:
null_counts = stars_df_clean.isna().sum()
null_counts

col1             0
_Glon            0
_Glat            0
_RAJ2000         0
_DEJ2000         0
              ... 
angDist          0
X                0
Y                0
Z                0
Unnamed: 0    1782
Length: 74, dtype: int64

Queda algunos valores nulos que también son derivados:
- e_FG, RFG, Gmag, e_Gmag, que provienen de nulos en FG
- e_FBP, RFBP, BPmag, e_BPmag, NBPcont, NBPblend, que provienen de nulos en FBP
- e_FRP, RFRP, RPmag, e_RPmag, NRPcont, NRPblend, que provienen de nulos en FRP
- E(BP/RP), que proviene de nulos en BP-RP

Estas columnas sin embargo sí son importantes, por lo que no podemos eliminarlas. Lo que sí podemos hacer es, teniendo en cuenta el pequeño porcentaje que representan respecto al total de filas, eliminar las filas. De nuevo, una sustitución por métricas estadísticas no sería adecuada.

In [39]:
stars_df_clean = stars_df_clean.dropna(subset=['FG'])
stars_df_clean = stars_df_clean.dropna(subset=['FBP'])
stars_df_clean = stars_df_clean.dropna(subset=['FRP'])
stars_df_clean = stars_df_clean.dropna(subset=['BP-RP'])
stars_df_clean = stars_df_clean.drop(columns=['Unnamed: 0'])
stars_df_clean.shape

(788304, 73)

In [40]:
null_counts = stars_df_clean.isna().sum()
null_counts

col1        0
_Glon       0
_Glat       0
_RAJ2000    0
_DEJ2000    0
           ..
E(BP-RP)    0
angDist     0
X           0
Y           0
Z           0
Length: 73, dtype: int64

Ya hemos limpiado el dataset de valores nulos

### 3.5 Eliminación de cúmulos con miembros insuficientes
Tras haber limpiado el dataset, puede que tengas cúmulos con un número insuficiente de estrellas como para obtener un buen rendimiento del modelo. Por ello eliminaremos definitivamente los cúmulos que se hayan quedado con menos de 20 miembros.

In [41]:
stars_df_clean['Cluster'] = stars_df_clean['Cluster'].str.strip()
print(stars_df_clean['Cluster'].unique())
print(len(stars_df_clean['Cluster'].unique()))

['NGC2682' 'NGC4147' 'NGC5466' 'NGC6791' 'NGC6811' 'NGC6819' 'NGC7789'
 'Pleiades' 'Teutsch51' 'NGC869' 'NGC1662' 'Basel11B' 'NGC1528' 'NGC2252'
 'Basel4' 'NGC1960' 'NGC6205' 'Waterloo2' 'NGC5272' 'Berkeley71'
 'Berkeley32' 'Berkeley17' 'Berkeley29' 'Berkeley53' 'Berkeley66' 'IC166'
 'King5' 'King7' 'M107' 'M13' 'M15' 'M2' 'M3' 'M35' 'M5' 'NGC2099'
 'NGC1746' 'NGC1039' 'NGC1647' 'NGC2183' 'NGC1912' 'NGC2281' 'NGC2420'
 'NGC2158' 'NGC2808' 'NGC6266' 'NGC2532' 'IC2395' 'NGC752' 'Mel25'
 'NGC2632' 'NGC2168' 'NGC884' 'NGC1851' 'NGC1904' 'NGC0104' 'M53' 'M67'
 'M71' 'M92' 'NGC1245' 'NGC1798' 'NGC188' 'NGC2243' 'NGC0362' 'NGC5139'
 'NGC3201' 'NGC5286' 'NGC5904' 'NGC6218' 'NGC6093' 'NGC6388' 'NGC6522'
 'NGC6254' 'NGC6293' 'NGC6441' 'NGC6541' 'NGC6624' 'NGC7078' 'NGC6656'
 'NGC6681' 'NGC6752' 'NGC7089' 'NGC1261' 'NGC7099' 'Berkeley86' 'IC1805'
 'IC2944' 'NGC457' 'NGC1502' 'NGC2244' 'NGC2362' 'NGC2384' 'NGC2422'
 'NGC2467' 'NGC3293' 'NGC4755' 'NGC6193' 'NGC7160' 'Trumpler14'
 'Trumpler16' 'NGC6

In [42]:
group_counts = stars_df_clean.groupby('Cluster').size().reset_index(name='Counts').sort_values(by='Counts')
group_counts

,Cluster,Counts
47,NGC2243,1
20,M35,1
103,Trumpler14,3
62,NGC4147,3
26,Mel25,4
...,...,...
87,NGC6712,34892
78,NGC6362,40229
71,NGC6139,52610
60,NGC3201,203335


In [43]:
groups_with_less_than_20 = group_counts[group_counts['Counts'] < 20]['Cluster'].tolist()
stars_df_clean = stars_df_clean[~stars_df_clean['Cluster'].isin(groups_with_less_than_20)]
stars_df_clean.shape

(788108, 73)

## 4. Inclusión de nuevas columnas
Creamos nuevas columnas calculadas a partir de las columnas existentes y que serán variables importantes en la clusterización:
- MG: magnitud absoluta, que da cuenta del brillo intrínseco de una estrella independientemente de su distancia
- B-R: color, que da cuenta del color intrínseco de una estrella corregido del efecto del medio interestelar. Esta variable estará relacionada con su temperatura, y por tanto con su masa, cualidad que las estrellas del cúmulo tendrán en común.

In [44]:
#MG=5-5*log(1000/Plx)-AG+Gmag
epsilon = 1e-10
positive_plx = np.abs(stars_df_clean['Plx']) + epsilon

# Calcular 'MG' asegurándose de no tener división por cero
stars_df_clean['MG'] = 5 * np.log10(1000 / positive_plx) - stars_df_clean['AG'] + stars_df_clean['Gmag']
#(B-R)=(BP-RP)-E(BP-RP)
stars_df_clean['B-R'] = stars_df_clean['BP-RP']-stars_df_clean['E(BP-RP)']

In [45]:
stars_df_clean.to_csv('../Samples/Clean/STARTSSample_EDA1.csv')

## 5 Factorización de variables categóricas y target
Se factorizan las variables categóricas y se guarda la tabla de equivalencia.

In [46]:
cat, num = EDA.explore(stars_df_clean)
print('Do it')

Rows: 788108
Columns: 75
+-------------+----------------+------------+-----------+---------------+
|             | Non-Null Count | Null Count | Data Type | Data Category |
+-------------+----------------+------------+-----------+---------------+
|    col1     |     788108     |     0      |   int64   |   Numerical   |
|    _Glon    |     788108     |     0      |  float64  |   Numerical   |
|    _Glat    |     788108     |     0      |  float64  |   Numerical   |
|  _RAJ2000   |     788108     |     0      |  float64  |   Numerical   |
|  _DEJ2000   |     788108     |     0      |  float64  |   Numerical   |
|   Cluster   |     788108     |     0      |  object   |  Categorical  |
|   e_RAdeg   |     788108     |     0      |  float64  |   Numerical   |
|   e_DEdeg   |     788108     |     0      |  float64  |   Numerical   |
|     Plx     |     788108     |     0      |  float64  |   Numerical   |
|    e_Plx    |     788108     |     0      |  float64  |   Numerical   |
|    RPlx    

In [47]:
stars_fz = EDA.factorize_categorical(stars_df_clean,cat)
stars_fz.to_csv('../Samples/Clean/STARTSSample_EDA1_fz.csv')

## 6 División por clusters
Ya que cda cúmulo tendrá unas características diferentes al resto, la búsqueda de outliers no tiene sentido hacerla de forma general, sino que debe hacerse teniendo en cuenta las estrellas de cda cúmulo esclusivamente. Para facilitar el proceso, dividiremos STARSSample en un dataset por cada cúmulo.

In [48]:
split_clusters(stars_fz)

In [49]:
print(stars_fz['Cluster_O'].unique())
print(len(stars_fz['Cluster_O'].unique()))

['NGC2682' 'NGC6791' 'NGC6811' 'NGC6819' 'NGC7789' 'Pleiades' 'NGC869'
 'NGC1662' 'Basel11B' 'NGC1528' 'NGC2252' 'Basel4' 'NGC1960' 'NGC6205'
 'Waterloo2' 'Berkeley71' 'Berkeley53' 'IC166' 'King5' 'King7' 'M107'
 'M13' 'M15' 'M2' 'M3' 'M5' 'NGC2099' 'NGC1746' 'NGC1039' 'NGC1647'
 'NGC2183' 'NGC1912' 'NGC2281' 'NGC2420' 'NGC2158' 'NGC2808' 'NGC6266'
 'NGC2532' 'IC2395' 'NGC752' 'NGC2632' 'NGC2168' 'NGC884' 'NGC1851'
 'NGC1904' 'NGC0104' 'M53' 'M67' 'M71' 'M92' 'NGC1245' 'NGC1798' 'NGC188'
 'NGC0362' 'NGC5139' 'NGC3201' 'NGC5286' 'NGC5904' 'NGC6218' 'NGC6093'
 'NGC6388' 'NGC6522' 'NGC6254' 'NGC6293' 'NGC6441' 'NGC6541' 'NGC6624'
 'NGC7078' 'NGC6656' 'NGC6681' 'NGC6752' 'NGC7089' 'NGC1261' 'NGC7099'
 'IC1805' 'IC2944' 'NGC2244' 'NGC2362' 'NGC3293' 'NGC4755' 'NGC6193'
 'Trumpler16' 'NGC6139' 'NGC6362' 'NGC6397' 'NGC6712' 'Pal13']
87


Ahora nos dividiremos en número de cumulos entre los tres para hacer el tratamiento de outliers en cada cúmulo. Una vez tengamos un dataset limpio de outliers de cada cúmulo, los concatenaremos para construir el dataset final.

- Rubén: NGC2682' 'NGC6791' 'NGC6811' 'NGC6819' 'NGC7789' 'Pleiades' 'NGC869'
 'NGC1662' 'Basel11B' 'NGC1528' 'NGC2252' 'Basel4' 'NGC1960' 'NGC6205'
 'Waterloo2' 'Berkeley71' 'Berkeley53' 'IC166' 'King5' 'King7' 'M107'
 'M13' 'M15' 'M2' 'M3' 'M5' 'NGC2099' 'NGC1746' 'NGC1039'
- Silvia: 'NGC1647' 'NGC2183' 'NGC1912' 'NGC2281' 'NGC2420' 'NGC2158' 'NGC2808' 'NGC6266'
 'NGC2532' 'IC2395' 'NGC752' 'NGC2632' 'NGC2168' 'NGC884' 'NGC1851'
 'NGC1904' 'NGC0104' 'M53' 'M67' 'M71' 'M92' 'NGC1245' 'NGC1798' 'NGC188'
 'NGC0362' 'NGC5139' 'NGC3201' 'NGC5286' 'NGC5904'
- Tatiana: 'NGC6218' 'NGC6093' 'NGC6388' 'NGC6522' 'NGC6254' 'NGC6293' 'NGC6441' 'NGC6541' 'NGC6624'
 'NGC7078' 'NGC6656' 'NGC6681' 'NGC6752' 'NGC7089' 'NGC1261' 'NGC7099'
 'IC1805' 'IC2944' 'NGC2244' 'NGC2362' 'NGC3293' 'NGC4755' 'NGC6193'
 'Trumpler16' 'NGC6139' 'NGC6362' 'NGC6397' 'NGC6712' 'Pal13'